<a href="https://colab.research.google.com/github/krnvz/server_client/blob/main/server/Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `CLAUDIO PINCHEIRA ROZAS`

![FASTAI (1).png](https://i.ibb.co/YTDrfMk/SERVER-1.png)

### Inicializaciones 

In [1]:
#@title Instalación de paquetes 
%%capture
!pip install opencv-python tensorflow
!pip install cvlib
!pip install uvicorn
!pip install fastapi
!pip install python-multipart
!pip install nest-asyncio
!pip install Pillow
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install colabcode

In [2]:
#@title librerias
from IPython.display import Image, display

from IPython.display import Image, display
import os
import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox
import io
import uvicorn
import numpy as np
import nest_asyncio
from enum import Enum
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import StreamingResponse

### Server

#### Directorios necesarios
Creación de directorios para almacenar las imágenes subidas al servidor y aquellas procesadas (con las detecciones)

In [3]:
# Creación de directorio para guardar las imagenes con detecciones
# ==============================================================================
location = '/content/'
dir_name = "images_with_boxes"
if not os.path.exists(location + dir_name):
    os.mkdir(location + dir_name)

In [4]:
# crear un directorio para guardar las imágenes que se vayan subiendo al servidor:
# ==============================================================================
dir_name = "images_uploaded"
location_imagen_uploaded = f"/content/" + dir_name
if not os.path.exists(location_imagen_uploaded):
    os.mkdir(location_imagen_uploaded)

#### Procesos del servidor

Asignación de una instancia de la clase FastAPI y definiciones de endpoints, con sus respectivos proceso tanto para identificar o  contar objetos en imágenes.

Funcionamiento de los endpoints:
* `/predict`: se le debe indicar un nivel de confianza, un modelo a utilizar y una imagen a evaluar. Retorna una imagen con la respectiva identificación y demarcación de un objeto y su etiqueta
* `/countObjects`:  se le debe indicar el nombre de algún objeto (en ingles y de forma singular), un nivel de confianza, un modelo a utilizar y por último la imagen a evaluar. Retorna la cantidad de objetos de interés, en la imagen.


In [5]:
# Asignamos una instancia de la clase FastAPI a la variable "app".
# ==============================================================================
app = FastAPI(title='Implementando un modelo de Machine Learning usando FastAPI')

# Enlistamos los modelos disponibles usando Enum. Útil cuando tenemos opciones predefinidas.
class Model(str, Enum):
    yolov3tiny = "yolov3-tiny"
    yolov3 = "yolov3"


# Usando @app.get("/") definimos un método GET para el endpoint / (que sería como el "home")
# ==============================================================================
@app.get("/")
def home():
    return "¡Felicitaciones! Tu API está funcionando según lo esperado. Anda ahora a http://localhost:8000/docs."


# Este endpoint maneja la lógica necesaria para detectar objetos.
# Requiere como entrada el modelo deseado y la imagen.
@app.post("/predict") 
def prediction(confidence, model: Model, file: UploadFile = File(...)):

    # 1. Validar el archivo de entrada
    filename = file.filename
    fileExtension = filename.split(".")[-1] in ("jpg", "jpeg", "png")
    if not fileExtension:
        raise HTTPException(status_code=415, detail="Tipo de archivo no soportado.")
    
    # 2. Transformar la imagen cruda a una imagen CV2
    
    # Leer la imagen como un stream de bytes
    image_stream = io.BytesIO(file.file.read())
    
    # Empezar el stream desde el principio (posicion cero)
    image_stream.seek(0)
    
    # Escribir el stream en un numpy array
    file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
    
    # Decodificar el numpy array como una imagen
    image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    
    
    # 3. Correr el modelo de detección de objetos
    
    # Correr la detección de objetos
    bbox, label, conf = cv.detect_common_objects(image, confidence= float(confidence) ,model=model)
    
    # Crear una imagen que contenga las cajas delimitadoras y etiquetas
    output_image = draw_bbox(image, bbox, label, conf)
    
    # Guardarla en un directorio del server
    cv2.imwrite(f'/content/images_uploaded/{filename}', output_image)
    
    
    # 4. Transmitir la respuesta de vuelta al cliente
    
    # Abrir la imagen para leerla en formato binario
    file_image = open(f'/content/images_uploaded/{filename}', mode="rb")
    
    # Retornar la imagen como un stream usando un formato específico
    return StreamingResponse(file_image, media_type="image/jpeg")

@app.post("/countObjects") 
def contadorObjetos(Objects , confidence , model: Model,file: UploadFile = File(...)):
    """
    Argumentos:
        Objects: string de objetos en ingles y de forma singular
        confidence: float, umbral de confianza para la asignación de la predicción
        model: modelo a usar
        file: (_io.BufferedReader) Archivo a subir, debe ser una imagen.
    """

    # PASO 1. Validar el archivo de entrada
    # ==============================================================================
    filename = file.filename #nombre del archivo
    fileExtension = filename.split(".")[-1] in ("jpg", "jpeg", "png") #verificar tipo de archivo
    if not fileExtension:
        raise HTTPException(status_code=415, detail="Tipo de archivo no soportado.")

    # PASO 2. Transformar la imagen cruda a una imagen CV2
    # ==============================================================================    
    image_stream = io.BytesIO(file.file.read())    # Leer la imagen como un stream de bytes
    image_stream.seek(0)   # Empezar el stream desde el principio (posicion cero)
    file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)   # Escribir el stream en un numpy array
    
    # Decodificar el numpy array como una imagen
    image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    

    # PASO 3. Correr el modelo de detección de objetos
    # ==============================================================================   
    # Correr la detección de objetos
    bbox, label, conf = cv.detect_common_objects(image, confidence= float(confidence) ,model=model)
    
    # Contar los objetos iguales a la palabra 'Object'
    count_obj = label.count(Objects)
    respuesta = 'Con un nivel de confianza de {}, se predice que en la imagen {} aparece {} veces el objeto {}'.format(float(confidence), filename, count_obj, Objects)
    return respuesta

####  Iniciar el servidor de código `colabcode`
La url aparecerá luego de correr la apps en el servidor y sera similar a: 
* `https://ad9aa39ca7d8.ngrok.io ====> ` (*enlace a modo de ejemplo, no usar*)

Aquel enlace debe ser usado en el colab del cliente, disponible en el siguiente link:

[Client.ipynb](https://colab.research.google.com/drive/1JXdOO3tzPpzgqm0JFM-V321cc7i5z9AW?usp=sharing)



In [6]:
from colabcode import ColabCode
server = ColabCode(port=65000, code=False)

In [8]:
print('Ahora el servidor esta funcionando, puedes ir al achivo de cliente \ny usar la primera URL acontinuación')
server.run_app(app=app)

Ahora el servidor esta funcionando, puedes ir al achivo de cliente 
y usar la primera URL acontinuación
Public URL: NgrokTunnel: "https://47c6-104-155-234-23.ngrok.io" -> "http://localhost:65000"


INFO:     Started server process [66]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:65000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [66]
